# Introduction

This notebook is intended to provide a pointer towards the various calculations (and figures).
These are each in their own notebook (with shared code in modules). The following links provide
an overview (hopefully in a sensible order).

* [Calculation of phase equilibrium](./phase_equilibrium.ipynb)
* [Boundary layer thickness and simple falling time](./falling_timescale.ipynb)
* [Nucleation and growth rate calculation](./nucleation_and_growth.ipynb)
* [An equilibrium time regime plot](./regime_plot.ipynb)
* [Calculation of particle size to grow the core from snow](./particle_size.ipynb)
* [Calculation of particle size from Marsh's equations](./marsh_growth_size.ipynb)
* [Growth of a falling particle](./falling_particle.ipynb)

There are also two sets of notes on CSD calculations 
[here](./CSD_cooling.ipynb) and [here with added CD notes](./CSD_cooling_cdnotes.ipynb) as well as some left
over stuff below.


## Notes

What defines equilibrium? 
1. Thermal Equilibrium: $T_S=T_L$ at the mutual interface (LR87 eqn 4.55). They also suggest at the start of section 4.3.1. that $F\Rightarrow\infty$ is required but we don't know what $F$ is. I think its in 4.45 and is something to do with entropy. 
2. Chemical equilibrium: $\mu_S = \mu_L$. Since $T$ and $p$ are equilibrated and flow near the surface of the solid is inhibited this is governed by diffusion in the chemical BLs on their side of the interface. 
3. Phase equilibrium: $\Lambda = \infty$. At phase equilibrium the rate of formation of solid will match the rate of destruction of solid because the total free energy will be minimised. That is to say, the volume of solid and the volume of liquid will be determined by applying the leaver rule to the phase diagram. This is what Sun is giving us below. The timescale is the sum of growth and nucleation timescales. The solid fraction is fixed and this is composed of $N$ particles: How long does it take to nucleate these $N$ particles and grow them to their equilibrium size? 

Notes for items (1) and (2) are in the notebook on  [falling timescales](./falling_timescale.ipynb). The notes on item (3) are below.

<ins>(3): Phase equilibrium</ins>: Equilibrium conditions give the volume fraction $V_E(T)$ of solid that corresponds to the state where chemical potentials in the solid and liquid are equal. However, there are an infinity of ways to partition $V_E$ between $N$ particles of radius $r$; the end-member cases are a large number of small particles and a small number of large particles. Additional information is needed. 

Here we will assume that the inner core is built from snow leaving the base of the F-layer. The snow is assumed to fall at the Stokes velocity $V_s$. We will assume that the inner core growth rate $d r_i / dt$ is given. Then the total volume change in time $d t$ is $d V / d t = 4\pi r_i^2 (d r_i /dt)$. The volume change (assuming spherical growth) is the number of particles flowing to the ICB at the Stokes velocity, which is $\frac{4\pi r^3}{3} N P V_s$, where the packing fraction $P=0.69$ for spheres. The two conditions corresponding to phase equilibrium and inner core growth are then

$$ V_E(T) = \frac{4 \pi r^3 N}{3} (1) , \quad \frac{4\pi r^3}{3} N P V_s = 4\pi r_i^2 \frac{d r_i}{dt} (2) ,$$

where $V_s$ is the generalised Stokes velocity as defined in Zhang and Xu (2002 - see below). Note that $r$ is not eliminated between these two equations because $V_s$ contains factors of $r$. 

To undertake the calculation there is the additional constraint that the particles must fall a maximum distance $\Delta r$ corresponding to the F-layer depth slower than the timescale required to reach phase equilibrium. If this is not true then the assumptions underlying the calculation (for a given IC growth rate) are incorrect. This comes into the calculation via $V_E$, which is actually the volume of solid in equilibrium divided by the total volume (i.e. the volume containing particles that can make it to the ICB). 

For a given $\Delta r$ and the $V_s$ obtained from equations (1) and (2) we can estimate the time $\tau_f$ it takes the particles to fall to the ICB. 

In fact equation (1) is actually $M$ constraints, one for each temperature $T(r)$. This is a problem because the second equation is only one constraint. To show this, consider that the F-layer is comprised of 2 levels, each with equilibrium volume, mean particle radius and mean particle number denoted $V_i$, $r_i$ and $N_i$ respectively, with $i = 1, 2$. The equations then become

$$ V_1 = \frac{4 \pi r_1^3 N_1}{3}$$ 
$$ V_2 = \frac{4 \pi r_2^3 N_2}{3}$$ 
$$\frac{4\pi r_1^3}{3} N_1 P V_{s1} + \frac{4\pi r_2^3}{3} N_2 P V_{s2} = 4\pi r_i^2 \frac{d r_i}{dt} ,$$

There are now 4 unknowns ($N_i$ and $R_i$) but only 3 equations. Further constraints are needed, but at present we are not sure what they are. 

<ins>(3a): Nucleation timescale</ins>:

The timescale to reach phase equilibrium, $\tau_E$, at radius $r$ with number of particles $N$ is approximately (might be a factor of 2 knocking about)

$$\tau_E(r,N) = \tau_N + \tau_G $$

where $\tau_N$ and $\tau_G$ are respectively the nucleation and growth timescales. 

$$ \tau_G = \frac{r}{V(T)}, \mbox{ where } V(T) = k(T) \left[1 - \exp\left(\frac{\Delta\mu}{k_B T}\right)  \right]$$

Problem here is that $\tau_N$ is the probability of nucleating 1 particle and we need the probability of nucleating $N$ particles. For $\tau_G$ we need to account for the fact that not all particles nucleate at the same time.  

For $\tau_N$ the critical radius is

$$ r_c \simeq - \frac{2\gamma T_m} {h_f  h_c(\delta T) \delta T}, $$

and is therefore a function of the interfacial energy $\gamma$ and the critical undercooling temperature $\delta T$: the closer the temperature to $T_m$ the larger the critical radius. The number of embryos of critical size $r_c$ per unit volume per unit time is

$$ I = I_0 \exp \left [ -\frac{16}{3}\frac{\pi \gamma^3 T_m^2}{k_{\rm B} T \delta T^2 h_f^2 [h_c(\delta T)]^2} \right ] $$
where $I_0$ is a prefactor which is related to the number of atoms per unit volume and an attempt frequency of bringing atoms together to form a solid embryo.

"Since they are at the top of the free energy barrier, on average approximately half of the embryos will re-melt, and the other half will grow and lead to a freezing event. The average waiting time to observe a freezing event, $\tau_N (units of s~m~$^3$), is therefore"

$$ \tau_N = \frac{1}{2 I_0} \exp \left [ \frac{16}{3}\frac{\pi \gamma^3 T_m^2}{k_{\rm B} T \delta T^2 h_f^2 [h_c(\delta T)]^2} \right ]. $$

Old notes of phase equilibrium: The timescale is the time taken for a liquid to reach phase equilibrium if it was instantly transported to defined temperature where solid and liquid were in equilibrium for a given particle size. To do this we would need to:
- Calculate the volume of solid for equilibrium from the free energy curves for the solid and liquid. If impurities partition between the solid and liquid this gets numerically complicated (see my ppv/pv notebook in LEMA!) but is really just a constrained minimisation problem. If we assume impurities only go in the liquid this should be much easer. For a pure single chemical component system phase equilibrium will always be 100% solid or 100% liquid.
- Work out how many particles of radius r we need to make the volume in (1). Work out how long it takes to form this volume of solid by nucleating the required number of particles and growing them by r. This is the timescale for phase equilibrium we need for the figure. At small r we will need many particles and so the time should be dominated by nucleation. At large radius, we will need fewer particles but they will need more time to grow, so time should be dominated by growth. In either case, the first thing we need is a function to give us the time to make a number of nuclei at a given temperature. We just need to add this line to the red line we have to get the anCalculate the volume of solid for equilibrium from the free energy curves for the solid and liquid. If impurities partition between the solid and liquid this gets numerically complicated (see my ppv/pv notebook in LEMA!) but is really just a constrained minimisation problem. If we assume impurities only go in the liquid this should be much easer. For a pure single chemical component system phase equilibrium will always be 100% solid or 100% liquid.
- Work out how many particles of radius r we need to make the volume in (1). Work out how long it takes to form this volume of solid by nucleating the required number of particles and growing them swer.

<ins>(3) revisited:</ins>
In LR world it is assumed that there is a slurry, i.e. a two-phase system, that is initially out of equilbrium. The question is: how long does it that this system to reach equilibrium. In this case the nucleation TS is irrelevant and only growth matters. Then the calculation is simply the Sun calculation. 

LR87: On pg 69 (bottom) they argue that "it would be more consistent to suppose that thermal and phase equilibrium proceed at a similar rate - see 3.2.2.". 

## OLD STUFF BELOW##

## Interface timescale. 

Here I implement equation 17 of Kirkpatrick (1974), "Kirkpatrick, R.J., 1975. Crystal growth from the melt: a review. American Mineralogist: Journal of Earth and Planetary Materials, 60(9-10), pp.798-814.". Below I refer to his equations as KX. 

- I assume that the growth rate is $da/dt$ where $a$ is particle radius
- Then time is $a/(da/dt)$
- I interpret this as the timescale required to grow the crystal to size a assuming that there are no diffusive hiderances to the process; the only obstacle seems to be the random detachment of particles from the surface according to K4.

- I take $f=1$
- Dario validates that K13 is a pretty good approximation; I have not included the correction in our eqn 2 here. 

The aassumption in K16 is checked in the cell kirk_growth17: it seems to be a bad approximation. Therefore we use eqn 14 in cell kirk_growth14. 

**However**

This model was found to be unsuitable because of the assumption that the rate constant can be described by a diffusion process (see eqn 9, which follows Turnbull and Cohen, 1960). It was replaced by the Sun et al study above. 

In [27]:
# Implement growth model of Kirkpatrick eqn 14, which does not invoke approximation K16
# Hc is latent heat
# Tl is liquidus
# eta is dynamic viscosity in Pa.s
def kirk_growth14(dT,Tl,eta,r):
    T = Tl - dT
    kB = 1.380649e-23 #J/K
    Rg = 8.314462
    f  = 1
    hf = 0.976e10
    Y  = (f*kB*T) / (3.0*np.pi*r**2*eta) * ( 1.0-np.exp( -(hf * dT)/(Rg*T*Tl) ) )
    return Y

In [28]:
# Implement growth model of Kirkpatrick eqn 17
# Hc is latent heat
# Tl is liquidus
# eta is dynamic viscosity in Pa.s
def kirk_growth17(dT,Tl,eta,r):
    kB = 1.380649e-23 #J/K
    Rg = 8.314462
    f  = 1
    hf = 0.976e10
    Y  = (f * kB) / (3.0 * np.pi * r**2 * eta) * (hf * dT) / (Rg * Tl)
    return Y

In [29]:
hf = 0.976e10 # From Dario
dT = 1.0      # 1 K undercooling
Tl = 6500.0   # Melting T of pure iron, high value
Tm_FeO = 5600.0
dT     = 200.0
tI     = np.zeros(len(r))

j = 0
for i in r:
    G = kirk_growth14(dT,Tm_FeO,0.01,i)
    tI[j] = i/G/secinyr
    j = j + 1

# 